# Test Field

In [1]:
import datetime
import re
from typing import List

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from sklearn.utils import resample

In [2]:
import final_func as fn
import importlib
importlib.reload(fn)

<module 'final_func' from '/Users/andrewmo/Documents/Docs - Jupiter/Projects/Class Individuals/2022Spring_Finals/final_func.py'>

In [3]:
# Load data
pit = pd.read_csv('data/pit_stops.csv')
results = pd.read_csv('data/results.csv')
status = pd.read_csv('data/status.csv')

In [4]:
# Process the data files
merge_df = fn.merge_data([pit, results, status])
merge_df = fn.process_data(merge_df)
df_dict = fn.pit_stop_group(merge_df)

In [5]:
def pit_stop_group(df: pd.DataFrame) -> dict:
    """
    group the records by the total number of pit stops of each racing record
    :param df: the merged and processed dataframe
    :return: a dictionary with total pit numbers as keys and dataframe of records as values
    """
    max_num = df['total_stops'].max()
    _df_dict = {}
    for i in range(1, max_num + 1):
        _df_dict[i] = df[df['total_stops'] == i][['stop', 'positionOrder', 'lap_prop']]
    return _df_dict

In [6]:
merge_df

,raceId,driverId,stop,lap,time_1,duration,milliseconds_1,resultId,constructorId,number,...,rank,fastestLapTime,fastestLapSpeed,statusId,status,total_laps,total_stops,lap_prop,abs_deviation,abs_deviation_mean
0,841,153,1,1,17:05:23,26.898,26898,20789,5,19,...,10,1:30.467,211.025,11,+1 Lap,58,3,0.017241,0.232759,0.195402
1,841,153,2,17,17:31:06,24.463,24463,20789,5,19,...,10,1:30.467,211.025,11,+1 Lap,58,3,0.293103,0.206897,0.195402
2,841,153,3,35,17:59:45,26.348,26348,20789,5,19,...,10,1:30.467,211.025,11,+1 Lap,58,3,0.603448,0.146552,0.195402
3,841,17,1,11,17:20:48,23.426,23426,20783,9,2,...,3,1:29.600,213.066,1,Finished,58,3,0.189655,0.060345,0.051724
4,841,17,2,26,17:44:29,22.520,22520,20783,9,2,...,3,1:29.600,213.066,1,Finished,58,3,0.448276,0.051724,0.051724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7913,1076,847,1,23,15:38:51,18.479,18479,25448,131,63,...,4,1:21.495,233.152,1,Finished,58,1,0.396552,0.103448,0.103448
7914,1076,4,1,39,16:05:24,18.796,18796,25462,214,14,...,2,1:20.846,235.024,11,+1 Lap,58,2,0.672414,0.339080,0.293103
7915,1076,4,2,53,16:25:28,18.394,18394,25462,214,14,...,2,1:20.846,235.024,11,+1 Lap,58,2,0.913793,0.247126,0.293103
7916,1076,825,1,39,16:05:30,19.111,19111,25459,210,20,...,15,1:23.071,228.729,11,+1 Lap,58,1,0.672414,0.172414,0.172414
